In [1]:
import edge_tts
import asyncio
import util    
import json
import sys
from IPython.display import Audio
from speakers.processors import load_preprocess,RvcProcessorData
from speakers.common.utils import get_abs_path
from omegaconf import OmegaConf

2023-08-26 22:08:55 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


from speakers.common.registry import registry rvc_speakers


/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


from speakers.common.registry import registry vits_to_voice
from speakers.common.registry import registry bark_to_voice
from speakers.common.registry import registry edge_to_voice
register_bootstrap runner_bootstrap_web
from speakers.common.registry import registry vits_voice_task
from speakers.common.registry import registry bark_voice_task
from speakers.common.registry import registry edge_voice_task
 Key with which the item will be registered {'device': 'cuda:0'}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center'

In [2]:
import logging
from speakers.processors.edge_to_voice import set_edge_to_voice_logger 
from speakers.processors.rvc_speakers_processor import set_rvc_speakers_logger
from speakers.processors.vits_to_voice import set_vits_to_voice_logger
from speakers.server.servlet.runner import set_server_runner_logger
from speakers.common.log import get_logger, set_log_level
set_log_level(level=logging.DEBUG) 
logger = get_logger('notebook')
set_rvc_speakers_logger(logger)
set_vits_to_voice_logger(logger)
set_edge_to_voice_logger(logger)
set_server_runner_logger(logger)

In [3]:
## 初始化 处理器

speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)
load_preprocess(config=config.get('preprocess'))
 

[notebook] limit text and audio length in huggingface spaces: False
[notebook] _load_voice_mode: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/vits/model/config.json


{'preprocess': [{'vits_processor': {'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}}, {'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}}, {'edge_processor': {'name': 'edge_to_voice'}}], 'tasks': [{'vits_voice_task': {'name': 'vits_voice_task', 'preprocess': [{'vits': {'processor': 'vits_processor', 'processor_name': 'VITS'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}, {'edge_voice_task': {'name': 'edge_voice_task', 'preprocess': [{'edge': {'processor': 'edge_processor', 'processor_name': 'EDGE'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}], 'bootstrap': [{'runner_bootstrap_web': {'name': 'runner_bootstrap_web', 'host': '0.0.0.0', 'port': 10001}}]}
{'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}


[notebook] Models loaded vits_to_voice
[notebook] Load hubert model/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/hubert_base.pt


{'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}


[notebook] Loaded hubert model
[notebook] Models Load:rvc_speakers
[notebook] rmvpe_path:/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/rmvpe.pt
[notebook] Loading model: yiqing
[notebook] Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/yiqing/config.json
[notebook] <All keys matched successfully>
[notebook] Loading model: bob
[notebook] Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/bob/config.json


...

gin_channels: 256 self.spk_embed_dim: 109


[notebook] <All keys matched successfully>
[notebook] Loading model: maomao
[notebook] Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/maomao/config.json


gin_channels: 256 self.spk_embed_dim: 109


[notebook] <All keys matched successfully>
[notebook] Models loaded:rvc_speakers, len:6


gin_channels: 256 self.spk_embed_dim: 109
{'name': 'edge_to_voice'}


In [4]:
from speakers.tasks import get_task,load_task
 

# 加载任务实例
speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)

load_task(config.get("tasks"))

{'preprocess': [{'vits_processor': {'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}}, {'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}}, {'edge_processor': {'name': 'edge_to_voice'}}], 'tasks': [{'vits_voice_task': {'name': 'vits_voice_task', 'preprocess': [{'vits': {'processor': 'vits_processor', 'processor_name': 'VITS'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}, {'edge_voice_task': {'name': 'edge_voice_task', 'preprocess': [{'edge': {'processor': 'edge_processor', 'processor_name': 'EDGE'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}], 'bootstrap': [{'runner_bootstrap_web': {'name': 'runner_bootstrap_web', 'host': '0.0.0.0', 'port': 10001}}]}


In [6]:
edge_voice_task = get_task("edge_voice_task")
edge_voice_task

In [11]:
from speakers.processors import get_processors,EdgeProcessorData
from speakers.tasks import EdgeVoiceFlowData, Runner
 


from collections import deque

edge_processor = get_processors('edge_processor')

rvc_processors = get_processors('rvc_processor')
 

tts_speaker = [
    '%s (%s)' % (
        s['FriendlyName'],
        s['Gender'], 
    )
    for s in edge_processor.tts_speakers_list
] 

tts_speaker_index = tts_speaker.index('Microsoft Xiaoxiao Online (Natural) - Chinese (Mainland) (Female)')
 
rate='+20%'
volume='-50%' 

# 创建一个 EdgeProcessorData 实例
edge_processor_data = EdgeProcessorData(text='我是你的情感导师，青春的时间转瞬即逝，秋水于海又岂有一只浮游',
                                      tts_speaker=tts_speaker_index,
                                      rate=rate,
                                      volume=volume)

loaded_models=[
        '%s - %s' % (
            m['metadata'].get('source', 'Unknown'),
            m['metadata'].get('name')
        )
        for m in rvc_processors.loaded_models
]
 


#变调(整数, 半音数量, 升八度12降八度-12)
f0_up_key=-1
f0_method =["pm", "harvest", "crepe", "rmvpe"]

#检索特征占比
index_rate = 0.9
#>=3则使用对harvest音高识别的结果使用中值滤波，数值为滤波半径，使用可以削弱哑音
filter_radius=1
#输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络
rms_mix_rate=1
#后处理重采样至最终采样率，0为不进行重采样
resample_rate=0

rvc_processor_data = RvcProcessorData(  
    model_index= 4,
    f0_up_key= f0_up_key,
    f0_method= f0_method[3],
    index_rate= index_rate,
    filter_radius= filter_radius,
    rms_mix_rate= rms_mix_rate,
    resample_sr= resample_rate 
)



# 创建一个 EdgeVoiceFlowData 实例，并将 EdgeProcessorData 实例作为参数传递
voice_flow_data = EdgeVoiceFlowData(edge=edge_processor_data,
                                rvc=rvc_processor_data)

# 创建 Runner 实例并传递上面创建的 BarkVoiceFlowData 实例作为参数
runner = Runner(
    task_id="1",
    flow_data=voice_flow_data
)
 



In [12]:
 
rate,output_audio = await edge_voice_task.dispatch(runner=runner)


loading rmvpe model


[notebook] npy: 0.011194467544555664s, f0: 0.7961552143096924s, infer: 0.17066693305969238s


In [13]:
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
print(rate)
# save audio to disk
write_wav(f"test.wav", rate, output_audio) 

# play text in notebook
Audio(output_audio, rate=rate)

40000
